In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.prune as prune
import torch.nn.functional as F

from scipy import io
import os

In [ ]:
mat_file = io.loadmat('/Users/goldenyoo/Library/Mobile Documents/com~apple~CloudDocs/BioCAS_prepare/Python_code/Data_center/one_dx/Calib_data_1.mat')


In [ ]:
K1 = mat_file['K1']
A1 = mat_file['A1']

K2 = mat_file['K2']
A2 = mat_file['A2']

# K1 = np.swapaxes(K1,0,2)
# K1 = np.swapaxes(K1,1,2)
# A1 = np.swapaxes(A1,0,2)
# A1 = np.swapaxes(A1,1,2)

# K2 = np.swapaxes(K2,0,2)
# K2 = np.swapaxes(K2,1,2)
# A2 = np.swapaxes(A2,0,2)
# A2 = np.swapaxes(A2,1,2)


Y1 = mat_file['Y1']
Y2 = mat_file['Y2']

In [ ]:
k1 = torch.FloatTensor(K1)

In [2]:
n_class = 9
n_hidden = 5
dtype = torch.float

class TextLSTM(nn.Module):
  def __init__(self):
    super(TextLSTM, self).__init__()

    self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden, dropout=0.3)
    self.fc = nn.Linear(n_hidden, 2)

  def forward(self, hidden_and_cell, X):
    X = X.transpose(0, 1)
    outputs, (h_n,c_n) = self.lstm(X, hidden_and_cell)
    outputs = h_n[-1]  # 최종 예측 Hidden Layer

    model = self.fc(outputs)  # 최종 예측 최종 출력 층
    return model

In [3]:
model = TextLSTM()


/Users/goldenyoo/miniforge3/envs/mac_cpu/lib/python3.8/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [8]:
# prune.random_unstructured(module_1, name="weight_ih_l0", amount=0.3)
print(list(model.named_parameters())[0][0])
print(list(model.named_parameters())[0][1].size())

lstm.weight_ih_l0
torch.Size([20, 9])


In [10]:
print(list(model.named_parameters())[1][0])
print(list(model.named_parameters())[1][1].size())

lstm.weight_hh_l0
torch.Size([20, 5])


In [4]:
list(model.state_dict().keys())

['lstm.weight_ih_l0',
 'lstm.weight_hh_l0',
 'lstm.bias_ih_l0',
 'lstm.bias_hh_l0',
 'fc.weight',
 'fc.bias']

In [ ]:
parameters_to_prune = (
    (model.lstm, 'weight_ih_l0'),
    (model.lstm, 'weight_hh_l0'),
    (model.lstm, 'bias_ih_l0'),
    (model.lstm, 'bias_hh_l0'),
    (model.fc, 'weight'),
    (model.fc, 'bias')
)

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.3,
)

In [ ]:
parameters_to_prune

In [ ]:
prune.remove(model.lstm, 'weight_ih_l0')
prune.remove(model.lstm, 'weight_hh_l0')
prune.remove(model.lstm, 'bias_ih_l0')
prune.remove(model.lstm, 'bias_hh_l0')
prune.remove(model.fc, 'weight')
prune.remove(model.fc, 'bias')